In [2]:
import gdal
import numpy as np
import xarray as xr

#### create your own population count manually

In [3]:
# read in population count
path = '/nfs/a68/earlacoa/population/count/v4.11/2015/'
population_tif = gdal.Open(
    path + 'gpw_v4_population_count_adjusted_to_2015_unwpp_country_totals_rev11_2015_15_min.tif'
)
population_count = population_tif.ReadAsArray()

# transform lat and lon
geo_transform = population_tif.GetGeoTransform()
population_x = np.linspace(
    geo_transform[0], 
    geo_transform[0] + geo_transform[1] * population_count.shape[1], 
    population_count.shape[1]
)
population_y = np.linspace(
    geo_transform[3], 
    geo_transform[3] + geo_transform[5] * population_count.shape[0], 
    population_count.shape[0]
)
population_xx, population_yy = np.meshgrid(population_x, population_y)

# replace negative fill values
population_count[population_count < 0] = 0.0

# older verions (< v4) count/lat is upside down
#population_count = np.flipud(population_count)

# save
np.savez_compressed(
    path + 'gpw_v4_population_count_adjusted_to_2015_unwpp_country_totals_rev11_2015_15_min.npz', 
    population_count=population_count,
    population_xx=population_xx,
    population_yy=population_yy
)

#### create xarray object

In [4]:
ds = xr.DataArray(
    population_count,
    dims=('lat', 'lon'),
    coords={
        'lat': population_y,
        'lon': population_x
    }
)

In [5]:
ds

<xarray.DataArray (lat: 720, lon: 1440)>
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)
Coordinates:
  * lat      (lat) float64 90.0 89.75 89.5 89.25 ... -89.25 -89.5 -89.75 -90.0
  * lon      (lon) float64 -180.0 -179.7 -179.5 -179.2 ... 179.5 179.7 180.0

In [7]:
ds.sum()

<xarray.DataArray ()>
array(7.34932e+09, dtype=float32)

In [8]:
ds.to_netcdf(path + 'gpw_v4_population_count_adjusted_to_2015_unwpp_country_totals_rev11_2015_15_min.nc')

#### or check this manual version is the same as the imported .npz version

In [16]:
def import_npz(npz_file, namespace):
    '''load all numpy arrays into global namespace'''
    '''ensure original arrays have the variable name you require'''
    data = np.load(npz_file)
    for var in data:
        if data[var].dtype == np.dtype('float64'):
            namespace[var] = data[var].astype('float32')
        else:
            namespace[var] = data[var]

In [17]:
# import pop_z_2015
import_npz('/nfs/a68/earlacoa/population/count/population-count-0.25deg.npz', globals())

In [34]:
print((pop_z_2015 == population_array).all())

True
